In [1]:
# ================================
# 국회 청원 상세보기 URL 크롤링 코드 (Jupyter Notebook 실행 가능)
# ================================

# 1️⃣ 필수 라이브러리 설치 및 로드
!pip install pandas beautifulsoup4 requests tqdm

import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

# 2️⃣ CSV 파일 로드
file_path = "청원_처리_현황.csv"

try:
    df_petition = pd.read_csv(file_path, encoding="cp949")
    print("✅ 기존 CSV 데이터 로드 성공!")
    print("📌 CSV 파일의 열 목록:", df_petition.columns.tolist())  # 열 목록 확인

    # '청원내용' 열이 없으면 추가
    if '청원내용' not in df_petition.columns:
        df_petition['청원내용'] = None
except Exception as e:
    print(f"❌ 기존 CSV 데이터 로드 실패: {e}")
    exit()

# 3️⃣ 상세보기 URL에서 청원 요지를 크롤링하는 함수
def get_petition_summary(url):
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")

            # ✅ '청원 요지'가 포함된 태그 찾기 (국회 사이트의 HTML 구조 기반)
            summary_element = soup.select_one("div.textType02#summaryContentDiv")

            if summary_element:
                return summary_element.text.strip()

            return None  # 청원 요지를 찾지 못한 경우
        else:
            print(f"❌ 요청 실패: {url}")
            return None
    except Exception as e:
        print(f"❌ 크롤링 오류: {e}")
        return None

# 4️⃣ 상세보기 URL을 크롤링하여 청원 요지 추가
for idx, row in tqdm(df_petition.iterrows(), total=len(df_petition), desc="크롤링 진행 중"):
    if pd.notna(row["상세보기URL"]) and (pd.isna(row["청원내용"]) or row["청원내용"] == ""):  # 기존 데이터가 없을 때만 크롤링
        df_petition.at[idx, "청원내용"] = get_petition_summary(row["상세보기URL"])
        time.sleep(1)  # 서버 부하 방지를 위해 1초 대기

# 5️⃣ 크롤링 완료된 데이터 저장
updated_file_path = "청원_처리_현황_크롤링완료.csv"
df_petition.to_csv(updated_file_path, index=False, encoding="utf-8-sig")
print(f"✅ 크롤링 완료! 저장된 파일: {updated_file_path}")

# 6️⃣ 크롤링 완료 후 학습 파일 실행 메시지 출력
print("🚀 크롤링이 완료되었습니다. 이제 `train.ipynb`를 실행하여 모델을 학습하세요!")


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
✅ 기존 CSV 데이터 로드 성공!
📌 CSV 파일의 열 목록: ['대', '청원번호', '청원제목', '청원인', '소개의원', '접수일자', '위원회회부일', '소관위', '의결결과', '의안구분', '상세보기URL', '상세보기URL2']


크롤링 진행 중: 100%|████████████████████████████████████████████████████████████████| 3730/3730 [1:42:48<00:00,  1.65s/it]

✅ 크롤링 완료! 저장된 파일: 청원_처리_현황_크롤링완료.csv
🚀 크롤링이 완료되었습니다. 이제 `train.ipynb`를 실행하여 모델을 학습하세요!
